<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Итоговый-вывод" data-toc-modified-id="Итоговый-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Итоговый вывод</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [1]:
import warnings

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

## Загрузка данных

In [2]:
try:
    df = pd.read_csv('/datasets/insurance.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
df.sample(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4209,0,21.0,24900.0,4,0
1428,0,43.0,27200.0,1,1
2520,1,21.0,46100.0,2,0
3598,1,29.0,23400.0,3,0
3462,0,35.0,26100.0,2,0


In [5]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
def show_col(df):
    for column in df.columns:
        print('Уникальные значения столбца "{}"'.format(column))
        print(df[column].sort_values().unique())
        print()
    print()
    print('Количество пропусков в каждом столбце:')
    print(df.isna().mean())

In [7]:
show_col(df)

Уникальные значения столбца "Пол"
[0 1]

Уникальные значения столбца "Возраст"
[18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62. 65.]

Уникальные значения столбца "Зарплата"
[ 5300.  6000.  7400.  8900.  9800. 10000. 10600. 10800. 11000. 11200.
 11300. 12200. 12900. 13000. 13200. 13300. 13400. 13500. 13800. 13900.
 14100. 14300. 14400. 14500. 14600. 14700. 15000. 15100. 15200. 15600.
 15700. 15900. 16000. 16200. 16300. 16400. 16500. 16600. 16700. 17000.
 17100. 17300. 17400. 17500. 17600. 17700. 17800. 17900. 18100. 18200.
 18300. 18400. 18600. 18700. 18800. 18900. 19000. 19100. 19200. 19300.
 19400. 19600. 19700. 19900. 20000. 20100. 20200. 20300. 20400. 20500.
 20600. 20700. 20800. 20900. 21000. 21100. 21200. 21300. 21400. 21500.
 21600. 21700. 21800. 21900. 22000. 22100. 22200. 22300. 22500. 22600.
 22700. 22800. 22900. 23000. 23100. 23200. 23300. 2340

Заменим названия столбцов и изменим тип столбцов "Возраст" и "Члены семьи":

In [8]:
df = df.rename(columns={'Пол': 'sex',
                        'Возраст': 'age',
                        'Зарплата': 'salary',
                        'Члены семьи': 'family_members',
                        'Страховые выплаты': 'insurance_payments'})

In [9]:
df['age'] = df['age'].astype(int)
df['salary'] = df['salary'].astype(int)

**Вывод:** Данные загружены и не имеют аномалий.

## Умножение матриц

In [10]:
features = df.drop('insurance_payments',axis=1)
target = df['insurance_payments']

features.shape, target.shape

((5000, 4), (5000,))

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.*

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

Проверим качество предсказания на матрице признаков $X$:

In [11]:
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
y = target
w = np.linalg.inv(X.T @ X) @ X.T @ y
display(w[1:])

model_X = LinearRegression()
model_X.fit(X, y)
print('R2 =',model_X.score(X, y))

array([ 7.92580563e-03,  3.57083050e-02, -1.70081903e-07, -1.35676627e-02])

R2 = 0.42494550308169177


Создадим обратимую матрицу $P$ и выведем ее обратную матрицу $P^{-1}$:

In [12]:
P = np.random.randint(1, 10,(X.shape[1],X.shape[1]))
np.linalg.inv(P)

array([[-0.03522367,  0.34202184, -0.04473406,  0.18198896, -0.35423271],
       [-0.0901726 ,  0.03557591, -0.0345192 ,  0.13255841, -0.01350241],
       [-0.02430433, -0.17400493,  0.1391335 , -0.04109428,  0.1122461 ],
       [ 0.06516379,  0.0172596 ,  0.03275801, -0.17001292,  0.13866385],
       [ 0.12469179, -0.15075731, -0.06164142,  0.02242574,  0.04731713]])

Умножим матрицу $X$ на обратимую матрицу $P$ и проверим качество предсказания:

In [13]:
Z = X @ P
w = np.linalg.inv(Z.T @ Z) @ Z.T @ y
display(w[1:])

model_Z = LinearRegression()
model_Z.fit(Z, y)
print('R2 = ',model_Z.score(Z, y))

array([ 0.08383504,  0.02486957, -0.06171339, -0.12102719])

R2 =  0.4249455030816992


**Ответ:** Не изменится.

**Обоснование:**

При умножении матрицы $X$ на обратимую матрицу $P$ получим новую матрицу $Z$:

$$
Z = XP
$$

Выясним чему будет равно предсказание $a_p$ и верктор весов $w_p$, используя матрицу $Z$:

$$
a_p = Zw_p
$$

$$
w_p = (Z^TZ)^{-1}Z^Ty
$$

Заменим вектор весов $w_p$ в формуле предсказания $a_p$:

$$
a_p = Z(Z^TZ)^{-1}Z^Ty
$$

Заменим матрицу $Z$ в формуле предсказания $a_p$:

$$
a_p = XP((XP)^TXP)^{-1}(XP)^Ty
$$

Для дальнейших преобразований понадобятся следующее свойство раскрытия скобок:

$$
(ABС)^{-1} = ((AB)С)^{-1} = ((AB)С)^{-1} = С^{-1}(AB)^{-1} = С^{-1}B^{-1}A^{-1}
$$

Проведем преобразование:

$$
a_p = XP((XP)^TXP)^{-1}(XP)^Ty =
$$
$$
= XPP^{-1}((XP)^TX)^{-1}(XP)^Ty
$$

Умножение обратимой матрицы $P$ на обратную матрицу $P^{-1}$ приведет к единичной матрице ($PP^{-1} = E$):

$$
a_p = XPP^{-1}((XP)^TX)^{-1}(XP)^Ty =
$$
$$
= XE((XP)^TX)^{-1}(XP)^Ty
$$

Теперь воспользуемся свойством транспонированной матрицы:

$$
(AB)^T = A^TB^T
$$

Умножение матрицы $X$ на единичную матрицу $E$ вернет нам матрицу $X$ ($XE = EX = X$). Проведем преобразование:

$$
a_p = XE((XP)^TX)^{-1}(XP)^Ty =
$$
$$
= X(P^TX^TX)^{-1}P^TX^Ty =
$$
$$
= X(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$

Умножение транспонированной обратимой матрицы $P^T$ на транспонированную обратную матрицу $(P^T)^{-1}$ приведет к единичной матрице ($P^T(P^T)^{-1} = E$):

$$
a_p = X(X^TX)^{-1}(P^T)^{-1}P^TX^Ty =
$$
$$
= X(X^TX)^{-1}EX^Ty =
$$
$$
= X(X^TX)^{-1}X^Ty =
$$
$$
= Xw = a
$$

В итоге мы вернулись к предсказанию $a$. Это означает, что умножение признаков на обратимую матрицу не повлияет на качество линейной регрессии.

Что касается весов линейной регрессии внашей задаче, то:

$$
w = (X^T X)^{-1} X^T y
$$

$$
w_P = ((XP)^T XP)^{-1} (XP)^T y =
$$
$$
= P^{-1}(P^TX^T X)^{-1} P^T X^T y =
$$
$$
= P^{-1}(X^T X)^{-1} (P^T){-1} P^T X^T y =
$$
$$
= P^{-1}(X^T X)^{-1} E X^T y =
$$
$$
= P^{-1}(X^T X)^{-1} X^T y =
$$
$$
= P^{-1} w
$$

Получается, что для того, чтобы зашифровать данные мы перемножаем матрицу признаков $X$ на обратимую функцию $P$, а вот веса $w$ мы умножаем уже на обратную функцию $P^{-1}$ для уравновешивания.

## Алгоритм преобразования

**Алгоритм**

Для защиты персональной информации клиентов умножим матрицу признаков $X$ на обратимую матрицу $P$, которая будет генерироваться случайним образом. Матрица $P$ должна иметь необходимую размерность $nxn$, где $n$ - количество признаков в матрице $X$ (в нашем случае $n = 4$). Определитель обратимой матрицы $P$ не должен быть равен $0$, т.е матрица $P$ должна быть невырожденной. **Если полученная случайным образом матрица $P$ будет иметь определитель равный $0$, то процесс дешифровки данных будет невозможен!** Полученную матрицу $Z$ стандартизируем в процессе построения модели.

$$
\left. X_{5000,4} =  
 \begin{pmatrix}
  x_{1,1} & x_{1,2} & x_{1,3} & x_{1,4} \\
  x_{2,1} & x_{2,2} & x_{2,3} & x_{2,4} \\
  \vdots  & \vdots  & \vdots & \vdots  \\
  x_{5000,1} & x_{5000,2} & x_{5000,3} & x_{5000,4}
 \end{pmatrix}
\middle.\hspace{0.5cm}
	\times
\middle.\hspace{0.5cm}
P_{4,4} =
\begin{pmatrix}
  p_{1,1} & p_{1,2} & p_{1,3} & p_{1,4} \\
  p_{2,1} & p_{2,2} & p_{2,3} & p_{2,4} \\
  p_{3,1} & p_{3,2} & p_{3,3} & p_{3,4} \\
  p_{4,1} & p_{4,2} & p_{4,3} & p_{4,4}
\end{pmatrix}
\right.\hspace{2cm}
detP \neq 0
$$

Этапы алгоритма:
1. Составление обратимой матрицы $P$. Если матрица необратимая повторяем п.1;
2. Проверка определителя обратимой матрицы $P$ на его неравенство $0$. Если определитель обратимой матрицы $P$ равен $0$, возвращаемся в п.1;
3. Перемножение матрицы признаков $X$ и обратимой матрицы $P$. Получаем матрицу $Z$, которую направляем для обучения модели.

**Обоснование**

Матрица $P$ должна иметь необходимую размерность $nxn$, где $n$ - количество признаков в матрице $X$ (в нашем случае $n = 4$). Таким образом матрица $Z$ будет иметь ту же размерность, что и матрица $X$. Далее направляем полученную матрицу для стандартизации и обучения модели линейной регрессии.

## Проверка алгоритма

Разделим данные на обучающую и тестовую выборки:

In [14]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=15243)

Построим модель линейной регрессии с данными из матрицы признаков $X$. Метрику R2 данной модели будем считать за константу:

In [15]:
model_X = LinearRegression()
model_X.fit(features_train, target_train)
R2_model_X = r2_score(target_test, model_X.predict(features_test))
print("R2 =", R2_model_X)

R2 = 0.40149290547870975


Проведем стандартизацию для матрицы признаков $X$, создадим модель и посчитаем R2:

In [16]:
model_X_scaled = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])
model_X_scaled.fit(features_train, target_train)
R2_model_X_scaled = r2_score(target_test,
                             model_X_scaled.predict(features_test))
print("R2 =", R2_model_X_scaled)

R2 = 0.40149290547870287


Стандартизация матрицы $X$ не изменила качество модели.

Для создания обратимой матрицы $P$ и последующего умножения ее на матрицу $X$ создадим функцию:

In [17]:
def encryption(X):
    n = X.shape[1]
    P = np.random.randint(1, 10, (n,n))
    det_P = np.linalg.det(P)
    while det_P == 0:
        P = np.random.randint(1, 10, (n,n))
        det_P = np.linalg.det(P)
    Z = X @ P
    return Z, P

Выведем первые 5 строк матрицы $X$, преобразуем ее в матрицу $Z$ и выведем ее, а так же выведем полученную обратимую матрицу $P$:

In [18]:
print('Матрица Х:')
display(features.head())
print('-'*60)
features_Z, matrix_P = encryption(features)
print('Матрица Z:')
display(features_Z.head())
print('-'*60)
print('Матрица P:')
matrix_P

Матрица Х:


,sex,age,salary,family_members
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


------------------------------------------------------------
Матрица Z:


,0,1,2,3
0,297978,198744,198655,99372
1,228421,152376,152282,76186
2,126261,84232,84174,42116
3,250403,166984,166938,83488
4,156854,104632,104571,52318


------------------------------------------------------------
Матрица P:


array([[2, 8, 3, 6],
       [9, 8, 6, 4],
       [6, 4, 4, 2],
       [7, 8, 6, 2]])

Разделим данные на обучающую и тестовую выборки:

In [19]:
(
    features_Z_train, features_Z_test, target_Z_train, target_Z_test
) = train_test_split(features_Z, target, test_size=0.25, random_state=15243)

Построим модель на основе матрицы признаков $Z$ и посчитаем R2:

In [20]:
model_Z = LinearRegression()
model_Z.fit(features_Z_train, target_Z_train)
R2_model_Z = r2_score(target_Z_test, model_Z.predict(features_Z_test))
print("R2 =", R2_model_Z)

R2 = 0.4014929054789096


Проведем стандартизацию для матрицы признаков $Z$, создадим модель и посчитаем R2:

In [21]:
model_Z_scaled = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])
model_Z_scaled.fit(features_Z_train, target_Z_train)
R2_model_Z_scaled = r2_score(target_Z_test,
                             model_Z_scaled.predict(features_Z_test))
print("R2 =", R2_model_Z_scaled)

R2 = 0.4014929054790194


Выведем информацию с результатами R2 для каждой модели:

In [22]:
rows = ['Линейная регрессия',
        'Линейная регрессия cо стандартизацией',
        'Линейная регрессия на преобразованных признаках',
        'Линейная регрессия на преобразованных признаках cо стандартизацией',]

result = pd.DataFrame(data= [R2_model_X,
                             R2_model_X_scaled,
                             R2_model_Z,
                             R2_model_Z_scaled],
                      columns=['R2'],
                      index=rows)
result

,R2
Линейная регрессия,0.401493
Линейная регрессия cо стандартизацией,0.401493
Линейная регрессия на преобразованных признаках,0.401493
Линейная регрессия на преобразованных признаках cо стандартизацией,0.401493


Для проверки шифрования напишем функцию дешифровки:

In [23]:
def decryption(Z, P):
    X = Z @ np.linalg.inv(P)
    for column in X.columns:
        X[column] = round(X[column], 0)
        X[column] = X[column].astype(int)
    X = X.rename(columns={0: 'sex',
                          1: 'age',
                          2: 'salary',
                          3: 'family_members'})
    return X

In [24]:
print('Матрица Х до шифрования:')
display(features.head())
print('-'*60)
print('Матрица Х после шифрования:')
decryp_Z = decryption(features_Z, matrix_P)
decryp_Z.head()

Матрица Х до шифрования:


,sex,age,salary,family_members
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


------------------------------------------------------------
Матрица Х после шифрования:


,sex,age,salary,family_members
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


Дешифровка проведена успешно.

## Итоговый вывод

В ходе работы было проделано:

- Загружены и изучены данные.
- Качество линейной регресии не изменилось от использования исходной матрицы и исходной матрицы, умноженную на обратимую.
- Создан алгоритм преобразования данных.
- Исследован алгоритм преобразования данных и проверена метрика R2 для данных без преобразования и с ним, а так же со стандартизацией и без нее.


По результатам работы использования матричных операций можно увидеть, что данные очень просто зашифровать от распознования, имея правильную матрицу.